In [ ]:
from bs4 import BeautifulSoup
from prefect import flow, task
from typing import List
import requests
import re

@task(retries=3, retry_delay_seconds=10)
def find_nike_price(url):
    k = requests.get(url).text
    soup = BeautifulSoup(k,'html.parser')
    price_string = soup.find('div', {"class":"product-price"}).text
    price_string = price_string.replace(' ','')
    price = int(re.search('[0-9]+',price_string).group(0))
    return price

@task
def inBudget(price, budget):
    return price <= budget

@task
def save(url: str, to: str = 'shoes_in_my_budget.html') -> None:
    with open(to, 'w') as f:
        f.write(url)

@flow
def nike_flow(url, budget):
    price = find_nike_price(url)
    
    if inBudget(price, budget):
        print('do something')

@flow
def parent_flow(urls: List[str]) -> None:
    for url in urls:
        nike_flow(url, budget)
        
if __name__ == "__main__":
    
    urls = ["https://www.nike.com/t/air-max-270-womens-shoes-Pgb94t/AH6789-601"]
    budget = 120
    
    parent_flow(urls)


: 